### Résumé prioritaire

- Fenêtres signalées : 3yr_1984.0_{A,B,C}, 3yr_2014.0_{A,B,C}, annual_2003.0_{A,C}, annual_2008.0_{A,B,C}, annual_2012.0_{A,B,C}, annual_2014.0_{A,C}, annual_2016.0_{A,B,C}.  
- Pattern commun : mêmes 5 features utilisées (temperature_celsius; humidity_percent; precipitation_mm; wind_speed_ms; urban_heat_island_intensity) et même ville signalée comme top offender : **Beijing_China_39.9042_116.4074**.  
- Observations typiques : prop_obs_outlier ≈ 0.0257 (≈2.6%), median_md ≈ 3.0, prop_cities_with_gt10pct_outliers = 0.05, max_city_outlier_frac ∈ [0.486, 0.538] — une seule ville concentre une large part des outliers.  
- Sensibilité : enlever la ville top‑offender provoque des deltas T_log notables (ex. −0.0786 pour 3yr_1984.0_A), donc la statistique globale est sensible à quelques villes.

---

### Chiffres clés à retenir

- Median Mahalanobis (median_md) ≈ 3.0 pour fenêtres concernées.  
- prop_obs_outlier ≈ 2.6% par fenêtre.  
- T_log exemples : 3yr_1984.0_A T_log = 0.06899 ; 3yr_2014.0_A T_log = 0.05121 ; annual_2008.0_A T_log = 0.14551 ; global T_log (logs) = −0.298824 (régime Divergence).  
- Transformations testées : winsor_precip, log1p_precip ; effets mesurés dans colonnes winsor_precip_T_log et log1p_precip_T_log du résumé.

---





### Vérifications immédiates (haut rendement / faible coût)

1. Confirmer la contribution de Beijing : ouvrir diagnostics_by_city pour une fenêtre (ex. results/.../3yr_1984.0_A/diagnostics_by_city_3yr_1984.0_A.csv) et lire n_obs, moyennes, et compte outliers par feature pour Beijing.  
2. Recalculer T_log en excluant Beijing pour une fenêtre représentative et mesurer delta_T_log_exclude_topcity. Si |delta| > 0.02, marquer comme haute sensibilité.  
3. Comparer winsor_precip_T_log vs log1p_precip_T_log dans tests_priority_summary.csv ; prioriser fenêtres où la transformation change fortement T_log.  
4. Vérifier dans summary_* quels features génèrent le plus d’outliers (precipitation_mm et urban_heat_island_intensity apparaissent souvent).

---


Commandes rapides utiles

Lister les fenêtres les plus sensibles (par |delta_T_log_exclude_topcity|) :

In [20]:
import pandas as pd
df = pd.read_csv("results/temporal_cv_atypical_windows/diagnostics/tests_priority_summary.csv")
df['delta_abs'] = df['delta_T_log_exclude_topcity'].abs()
df.sort_values('delta_abs', ascending=False).head(10)[['tag','window_center','pipeline','T_log','exclude_topcity_T_log','delta_T_log_exclude_topcity']]


,tag,window_center,pipeline,T_log,exclude_topcity_T_log,delta_T_log_exclude_topcity
13,annual_2012.0_C,2012.0,C,0.052576,-0.147700,-0.200276
12,annual_2012.0_B,2012.0,B,0.052576,-0.147700,-0.200276
11,annual_2012.0_A,2012.0,A,0.052576,-0.147700,-0.200276
9,annual_2008.0_B,2008.0,B,0.145512,0.052835,-0.092677
8,annual_2008.0_A,2008.0,A,0.145512,0.052835,-0.092677
10,annual_2008.0_C,2008.0,C,0.145512,0.052835,-0.092677
1,3yr_1984.0_B,1984.0,B,0.068987,-0.009656,-0.078643
0,3yr_1984.0_A,1984.0,A,0.068987,-0.009656,-0.078643
2,3yr_1984.0_C,1984.0,C,0.068987,-0.009656,-0.078643
16,annual_2016.0_A,2016.0,A,0.029690,-0.023059,-0.052749


Recalculer T_log sans Beijing (pseudo) :

In [21]:
w = "results/temporal_cv_atypical_windows/temporal_cv_atypical_period_window_3yr_1984.00_A_raw.csv"
dfw = pd.read_csv(w)
df_no_beijing = dfw[dfw['city_key'] != "Beijing_China_39.9042_116.4074"]
# Agréger par city_key, standardiser, calculer d_part/d_pca90 comme dans votre routine, puis T_log = (d_est-4)*ln(n_eff)


Cellule Python automatisée — recalculer T_log sans la ville spécifiée (batch)

 La cellule :

lit tests_priority_summary.csv et sélectionne les fenêtres à traiter (par défaut top 10 par |delta_T_log_exclude_topcity|) ;

pour chaque fenêtre : charge le raw CSV listé dans l’index enrichi, agrège par city_key (moyennes temporelles), standardise, calcule d_part, d_pca90, d_est et T_log original et T_log recalculé en excluant la ville top_offender ;

sauvegarde un CSV résumé comparatif et pour chaque fenêtre un petit dossier contenant plots (heatmap per-city×feature medians, timeseries du top_city pour precipitation_mm si présente) ;

paramètres configurables en haut de la cellule.

In [22]:
# Cellule : recalcul automatique T_log sans la ville top_offender (batch)
import os, re, math, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Configurable
IDX_ENR = 'results/temporal_cv_atypical_windows/temporal_cv_atypical_periods_windows_index_enriched.csv'
PRIO_SUM = 'results/temporal_cv_atypical_windows/diagnostics/tests_priority_summary.csv'
OUT_DIR = 'results/temporal_cv_atypical_windows/diagnostics/tlog_exclude_city_results'
TOPN = 10               # nombre de fenêtres à traiter (par delta absolu)
DEFAULT_FEATURES = ['temperature_celsius','humidity_percent','precipitation_mm','wind_speed_ms','urban_heat_island_intensity']
os.makedirs(OUT_DIR, exist_ok=True)

def safe_tag(s): return re.sub(r'[^0-9A-Za-z_.-]', '_', str(s))

def compute_d_estimate_from_X(X):
    # X : (cities x features) already standardized float ndarray
    if X.shape[0] < 2:
        return np.nan, np.nan, np.nan
    cov = np.cov(X, rowvar=False)
    eigvals = np.linalg.eigvalsh(cov)
    eigvals = np.maximum(eigvals, 0.0)
    sum_eig = np.sum(eigvals)
    d_part = 0.0 if sum_eig <= 0 else (sum_eig**2) / np.sum(eigvals**2)
    pca = PCA(n_components=min(X.shape[0], X.shape[1])).fit(X)
    cumvar = np.cumsum(pca.explained_variance_ratio_)
    d_pca90 = int(np.searchsorted(cumvar, 0.90) + 1) if cumvar[-1] >= 0.90 else pca.n_components_
    d_est = float((d_part + d_pca90) / 2.0)
    return d_part, d_pca90, d_est

def compute_T_log(d_est, n_eff):
    n_eff = max(2, int(n_eff))
    return (d_est - 4.0) * np.log(n_eff)

# Load priority table (must exist)
if not os.path.exists(PRIO_SUM):
    raise FileNotFoundError(f"Priority summary introuvable: {PRIO_SUM}")

prio = pd.read_csv(PRIO_SUM)
# ensure delta exists
if 'delta_T_log_exclude_topcity' not in prio.columns:
    raise RuntimeError("Colonne delta_T_log_exclude_topcity absente du fichier priority summary")

# pick top windows by absolute delta (or all if fewer)
prio['delta_abs'] = prio['delta_T_log_exclude_topcity'].abs().fillna(0.0)
sel = prio.sort_values('delta_abs', ascending=False).head(TOPN)

results = []
for _, row in sel.iterrows():
    tag = row['tag']
    window_type = row.get('window_type')
    window_center = row.get('window_center')
    pipeline = row.get('pipeline')
    raw_candidates = []
    # try to find raw csv path: check index enriched first
    if os.path.exists(IDX_ENR):
        idx = pd.read_csv(IDX_ENR)
        match = idx[(idx['window_type']==window_type) & (idx['window_center']==float(window_center)) & (idx['pipeline']==pipeline)]
        if not match.empty and 'raw_csv' in match.columns:
            raw_candidates.append(match.iloc[0]['raw_csv'])
    # fallback heuristics
    raw_candidates.extend(glob.glob(os.path.join('results','temporal_cv_atypical_windows',f'*{tag}*raw.csv')))
    raw_fp = next((p for p in raw_candidates if p and os.path.exists(p)), None)
    if raw_fp is None:
        results.append({'tag': tag, 'status': 'no_raw_found'})
        continue

    df = pd.read_csv(raw_fp)
    if 'city_key' not in df.columns:
        agg_cols = [c for c in ['city','country','latitude','longitude'] if c in df.columns]
        df['city_key'] = df[agg_cols].astype(str).agg('_'.join, axis=1) if agg_cols else 'unknown'

    # detect features to use
    num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in ['year','month','n_rows']]
    features = [c for c in DEFAULT_FEATURES if c in num_cols] or (num_cols[:min(5,len(num_cols))] if num_cols else [])
    if not features:
        results.append({'tag': tag, 'status': 'no_numeric_features'})
        continue

    # aggregate per-city (median or mean; use mean for consistency)
    city_df = df.groupby('city_key')[features].mean().reset_index()
    # drop cities with any NaN in the selected features
    city_df_clean = city_df.dropna(subset=features).reset_index(drop=True)
    n_cities = city_df_clean.shape[0]
    if n_cities < 2:
        results.append({'tag': tag, 'status': 'not_enough_cities', 'n_cities': n_cities})
        continue

    scaler = StandardScaler()
    X = scaler.fit_transform(city_df_clean[features].astype(float).values)
    d_part, d_pca90, d_est = compute_d_estimate_from_X(X)
    T_log_orig = compute_T_log(d_est, X.shape[0])

    top_city = row.get('top_city')
    if not top_city or top_city not in city_df_clean['city_key'].values:
        # try best-effort: pick city with max outlier fraction in per-city diagnostics if present
        # fallback to city with largest n_obs in raw df
        if 'city_key' in df.columns:
            counts = df['city_key'].value_counts()
            top_city = counts.index[0] if not counts.empty else None

    # compute per-city medians matrix for heatmap
    heatmap_mat = city_df_clean.set_index('city_key')[features]

    # Recompute excluding top city if possible
    exclude_T_log = np.nan
    delta = np.nan
    d_part_ex = d_pca90_ex = d_est_ex = np.nan
    if top_city and top_city in city_df_clean['city_key'].values:
        city_df_ex = city_df_clean[city_df_clean['city_key'] != top_city].reset_index(drop=True)
        if city_df_ex.shape[0] >= 2:
            Xex = scaler.fit_transform(city_df_ex[features].astype(float).values)
            d_part_ex, d_pca90_ex, d_est_ex = compute_d_estimate_from_X(Xex)
            exclude_T_log = compute_T_log(d_est_ex, Xex.shape[0])
            delta = exclude_T_log - T_log_orig

    # Save summary entry
    res_row = {
        'tag': tag, 'window_type': window_type, 'window_center': window_center, 'pipeline': pipeline,
        'raw_csv': raw_fp, 'n_cities': n_cities, 'features': ';'.join(features),
        'top_city': top_city, 'T_log_orig': float(T_log_orig), 'T_log_exclude_topcity': float(exclude_T_log) if not np.isnan(exclude_T_log) else np.nan,
        'delta_T_log_exclude_topcity': float(delta) if not np.isnan(delta) else np.nan,
        'd_part': float(d_part), 'd_pca90': int(d_pca90), 'd_est': float(d_est),
        'd_part_ex': float(d_part_ex) if not np.isnan(d_part_ex) else np.nan,
        'd_pca90_ex': int(d_pca90_ex) if not np.isnan(d_pca90_ex) else np.nan,
        'd_est_ex': float(d_est_ex) if not np.isnan(d_est_ex) else np.nan,
        'status': 'ok'
    }
    results.append(res_row)

    # Plots folder for this window
    fig_dir = os.path.join(OUT_DIR, safe_tag(tag))
    os.makedirs(fig_dir, exist_ok=True)

    try:
        # heatmap of city medians (z-scored per feature)
        hm = heatmap_mat.copy().astype(float)
        for col in hm.columns:
            colv = hm[col]
            if colv.dropna().shape[0] >= 2 and not np.isclose(colv.std(ddof=1), 0.0):
                hm[col] = (colv - colv.mean()) / colv.std(ddof=1)
            else:
                mn = colv.min(); mx = colv.max()
                hm[col] = 0 if np.isclose(mx, mn) else (colv - mn) / (mx - mn)
        plt.figure(figsize=(8, max(4, min(30, hm.shape[0]*0.25))))
        plt.imshow(hm.values, aspect='auto', cmap='vlag', interpolation='nearest')
        plt.colorbar(label='normalized median')
        plt.yticks(range(len(hm.index)), hm.index)
        plt.xticks(range(len(hm.columns)), hm.columns, rotation=45, ha='right')
        plt.title(f'Heatmap medians per city - {tag}')
        plt.tight_layout()
        plt.savefig(os.path.join(fig_dir, f'{safe_tag(tag)}_heatmap.png'), dpi=150)
        plt.close()
    except Exception:
        pass

    try:
        # top city timeseries for precipitation_mm if present
        if 'precipitation_mm' in features and top_city in df['city_key'].unique():
            df_city = df[df['city_key']==top_city].sort_index()
            plt.figure(figsize=(8,2.5))
            plt.plot(df_city['precipitation_mm'].values, marker='o', markersize=3, linestyle='-')
            plt.title(f'{tag} - precipitation_mm timeseries - {top_city}')
            plt.tight_layout()
            plt.savefig(os.path.join(fig_dir, f'{safe_tag(tag)}_topcity_precip_timeseries.png'), dpi=150)
            plt.close()
    except Exception:
        pass

# Write results CSV
out_csv = os.path.join(OUT_DIR, 'tlog_exclude_topcity_comparison.csv')
pd.DataFrame(results).to_csv(out_csv, index=False)
print("Résultats écrits :", out_csv)
print("Plots par fenêtre sous :", OUT_DIR)


Résultats écrits : results/temporal_cv_atypical_windows/diagnostics/tlog_exclude_city_results\tlog_exclude_topcity_comparison.csv
Plots par fenêtre sous : results/temporal_cv_atypical_windows/diagnostics/tlog_exclude_city_results


### Résumé rapide des résultats (tlog_exclude_topcity_comparison.csv)

- **Top city identifiée** : Beijing_China_39.9042_116.4074 pour toutes les fenêtres listées.  
- **Effet d’exclusion** : pour chaque fenêtre, l’exclusion de la top city fait chuter T_log (delta_T_log_exclude_topcity < 0 dans tous les cas).  
- **Amplitude de l’effet** : les baisses de T_log vont d’environ **-0.05** à **-0.20** (ex. annual_2012 Δ ≈ -0.20; annual_2008 Δ ≈ -0.093; 3yr_1984 Δ ≈ -0.079; annual_2016 Δ ≈ -0.053).  
- **Impact sur l’estimation de dimension d** : la valeur finale d_est diminue légèrement après exclusion (typ. baisse de ~0.03–0.07), indiquant que la top city contribue à accroître l’estimation effective de la dimension.  
- **Statut de ces lignes** : toutes marquées "ok".

---

### Interprétation actionable (conclusions directes)
- Beijing exerce une influence positive sur T_log dans ces sous-ensembles; son retrait rend le diagnostic plus négatif (pousse vers la divergence).  
- Les changements de d_est sont modestes mais cohérents : la présence de la top city augmente légèrement la complexité effective perçue du jeu de villes.  
- L’effet n’est pas catastrophique (pas de renversement de signe massif sauf réduction notable), mais suffisamment large sur certaines fenêtres (ex. annual_2012) pour mériter investigation.


---

### Résumé court (une ligne)
L’exclusion de Beijing rend systématiquement T_log plus négatif et réduit légèrement d_est ; Beijing est un contributeur structurant du signal dans ces fenêtres — action recommandée : quantifier l’influence LOO complète, décomposer par feature, et tester estimateurs plus robustes avant décision finale.

Objectifs précis de l’exécution LOO étendue

Pour chaque fenêtre flaggée, recalculer T_log en retirant chaque ville à tour de rôle.

Produire un CSV résumé classant les villes par delta moyen (T_log_without_city − T_log_full), delta absolu, et nombre d’occasions où la suppression inverse le régime.

Sauvegarder pour chaque fenêtre : top 5 villes influentes + un plot montrant la distribution LOO de T_log et la position du T_log complet.

Résultat pratique : shortlist des villes à downweight/exclure/tester avec corrections de features.

Cellule Python — LOO étendue, classement d’influence et plots

 Elle parcourt les fenêtres listées dans tests_priority_summary.csv (ou index enrichi), effectue LOO par ville, écrit un CSV classement et exporte plots par fenêtre.